# 02 — Preprocessing (NumPy-only)

Mục tiêu: đọc `AB_NYC_2019.csv` **không dùng pandas**, làm sạch/biến đổi tối thiểu, tạo đặc trưng, chuẩn hoá Z-score, rồi lưu `data/processed/airbnb_2019_preprocessed.npz` để dùng cho `03_modeling`.

**Quy ước**  
- Xử lý thiếu:
  - `reviews_per_month`: điền `0.0` (nhiều listing chưa có review → hợp lý gán 0).
  - Numeric khác: điền **median** cột.
- Ngoại lai numeric: **clip** theo percentiles (mặc định 1–99%).
- Mục tiêu: `y = log1p(price)`.
- Phân loại: one-hot `neighbourhood_group`, `room_type`.
- Chuẩn hoá: **Z-score** trên toàn bộ X của HW02 (đơn giản cho chấm tự động).


In [ ]:
# Thêm project root (thư mục chứa 'src/') vào sys.path
from pathlib import Path
import sys

ROOT = Path.cwd()
while not (ROOT / "src").is_dir() and ROOT.parent != ROOT:
    ROOT = ROOT.parent

sys.path.append(str(ROOT))
print("PROJECT ROOT:", ROOT)

## 1. Khởi tạo & import

In [ ]:
import numpy as np
from pathlib import Path
from src.data_processing import load_csv_genfromtxt, preprocess_and_save  # dùng lại func đã build sẵn

# Đường dẫn dữ liệu
DATA_DIR = ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

CSV_PATH = RAW_DIR / "AB_NYC_2019.csv"   # đổi tên nếu bạn dùng file khác
OUT_NPZ  = PROCESSED_DIR / "airbnb_2019_preprocessed.npz"

print("CSV path:", CSV_PATH.exists(), CSV_PATH)
print("OUT npz :", OUT_NPZ)

## 2) Đọc CSV bằng NumPy (genfromtxt)


In [ ]:
header, data = load_csv_genfromtxt(CSV_PATH)

print("Columns:", len(header))
print("First 10 cols:", header[:10].tolist())
print("Shape data:", data.shape)

In [ ]:
# Lập map tên cột -> index để truy cập nhanh
col2idx = {name: i for i, name in enumerate(header.tolist())}

## 3) Helpers (NumPy-only)

In [ ]:
def _safe_to_float(a: np.ndarray) -> np.ndarray:
    # ép object -> float, chuỗi rỗng/None -> NaN
    out = a.astype(object)
    v = np.vectorize(lambda x: np.nan if (x is None or x == "" or (isinstance(x, str) and x.strip()=="" )) else x)
    out = v(out)
    return out.astype(float)

def _one_hot(cat: np.ndarray):
    uniq = np.unique(cat.astype(object))
    idx = {u: i for i, u in enumerate(uniq)}
    O = np.zeros((cat.shape[0], len(uniq)), dtype=float)
    for r, v in enumerate(cat):
        O[r, idx[v]] = 1.0
    names = [f"cat=={u}" for u in uniq]
    return O, uniq, names

def clip_outliers(x: np.ndarray, p_lo=1.0, p_hi=99.0) -> np.ndarray:
    lo, hi = np.nanpercentile(x, [p_lo, p_hi])
    return np.clip(x, lo, hi)

## 4) Làm sạch cột numeric + thống kê nhanh
- `reviews_per_month`: điền `0.0` (chưa có review)
- Numeric khác: điền **median**
- Clip ngoại lai P1–P99 để giảm ảnh hưởng đuôi
- In vài thống kê để 

In [ ]:
NUMERIC_COLS = (
    "minimum_nights",
    "number_of_reviews",
    "reviews_per_month",
    "calculated_host_listings_count",
    "availability_365",
)

def describe_quick(x: np.ndarray, name: str):
    print(f"[{name}] miss={int(np.isnan(x).sum())} "
          f"| mean={np.nanmean(x):.3f} std={np.nanstd(x):.3f} "
          f"| p1={np.nanpercentile(x,1):.3f} p50={np.nanmedian(x):.3f} p99={np.nanpercentile(x,99):.3f}")

feats = []
feat_names = []

for name in NUMERIC_COLS:
    col = _safe_to_float(data[:, col2idx[name]])

    # Fill NA
    if name == "reviews_per_month":
        col = np.where(np.isnan(col), 0.0, col)    # hợp lý gán 0 cho listing chưa có review
    else:
        med = np.nanmedian(col)
        col = np.where(np.isnan(col), med, col)

    print("Before clip:", end=" "); describe_quick(col, name)
    col = clip_outliers(col, 1.0, 99.0)
    print("After  clip:", end=" "); describe_quick(col, name); print()

    feats.append(col.reshape(-1, 1))
    feat_names.append(name)

## 4) Biến mục tiêu & đặc trưng phân loại

- Mục tiêu: `y = log1p(price)` để giảm lệch phải  
- Phân loại: one-hot cho `neighbourhood_group`, `room_type` 

In [ ]:
# y = log1p(price)
price = _safe_to_float(data[:, col2idx["price"]])
y = np.log1p(price)  # biến mục tiêu

# one-hot categorical
CAT_COLS = ("neighbourhood_group", "room_type")
for name in CAT_COLS:
    cat = data[:, col2idx[name]].astype(object)
    O, uniq, names = _one_hot(cat)
    feats.append(O)
    # đổi prefix name cho rõ
    names = [f"{name}=={u}" for u in uniq]
    feat_names.extend(names)

# Gom X
X = np.hstack(feats)
feature_names = np.array(feat_names, dtype=object)

X.shape, y.shape, feature_names.shape

## 5) Chuẩn hoá Z-score & lưu `.npz`

In [ ]:
mean = X.mean(axis=0, dtype=float)
std  = X.std(axis=0, dtype=float)
std  = np.where(std == 0, 1.0, std) # tránh chia 0

X_std = (X - mean) / std

np.savez_compressed(
    OUT_NPZ,
    X=X_std,
    y=y.astype(float),
    feature_names=feature_names,
    stats_mean=mean,
    stats_std=std
)

print("Saved ->", OUT_NPZ, "| shapes:", X_std.shape, y.shape)

## 6) Sanity check

In [ ]:
bundle = np.load(OUT_NPZ, allow_pickle=True)
Xc = bundle["X"]; yc = bundle["y"]
names = bundle["feature_names"]

print("Shapes:", Xc.shape, yc.shape)
print("NaN in X:", int(np.isnan(Xc).sum()), "| NaN in y:", int(np.isnan(yc).sum()))
assert np.isfinite(Xc).all(), "X contains NaN/Inf"
assert np.isfinite(yc).all(), "y contains NaN/Inf"

In [ ]:
# (Optional) xem top |corr| với y thô (price) chỉ để tham khảo nhanh
# Lưu ý: y đã log1p khi train; đây chỉ là xem mối liên hệ sơ bộ.
with np.errstate(invalid="ignore"):
    # corr với y (log1p(price)) theo Pearson giữa từng cột và y
    xy = []
    for j in range(Xc.shape[1]):
        v = Xc[:, j]
        if np.std(v) == 0:
            c = 0.0
        else:
            c = np.corrcoef(v, yc)[0,1]
        xy.append(c)
xy = np.array(xy)
top_idx = np.argsort(-np.abs(xy))[:10]
print("\nTop 10 |corr| với y (log1p(price)):")
for j in top_idx:
    print(f"{names[j]:<40s} corr={xy[j]: .4f}")

## 7) (Optional) Command line (nếu muốn dùng API có sẵn)

Ở notebook này đã làm từng bước để rõ ràng.
Nhưng cũng có thể dùng lại `src.data_processing.preprocess_and_save(...)` (đã implement sẵn) để sinh `.npz` trực tiếp:

```python
# Tạo lại .npz bằng API có sẵn:
# from src.data_processing import preprocess_and_save
# preprocess_and_save(CSV_PATH, OUT_NPZ)